In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Question Answering with Generative Models on Vertex AI

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/language/examples/prompt-design/question_answering.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/examples/prompt-design/question_answering.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/blob/main/language/examples/prompt-design/question_answering.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
</table>


## Overview

Large language models can be used for various natural language processing tasks, including question-answering (Q&A). These models are trained on a vast amount text data and can generate high-quality responses to a wide range of questions. One thing to note here is that most models have cutoff dates regarding their knowledge, and asking anything too recent might yield an incomplete, imaginative or incorrect answer (i.e. a hallucination).

This notebook covers the essentials of prompts for answering questions using a generative model. In addition, it showcases the `open domain` (knowledge available on the public internet) and `closed domain` (knowledge that is more private - typically enterprise or personal knowledge).

Learn more about prompt design in the [official documentation](https://cloud.google.com/vertex-ai/docs/generative-ai/text/text-overview#prompt_structure).

### Objective

By the end of the notebook, you should be able to write prompts for the following:

* **Open domain** questions:
    * Zero-shot prompting
    * Few-shot prompting


* **Closed domain** questions:
    * Providing custom knowledge as context
    * Instruction-tune the outputs
    * Few-shot prompting

## Getting Started

### Install Vertex AI SDK

In [1]:
!pip install google-cloud-aiplatform --upgrade --user

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.0/321.0 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 24.4 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


**Colab only:** Uncomment the following cell to restart the kernel or use the button to restart the kernel. For Vertex AI Workbench you can restart the terminal using the button on top.

In [1]:
# # Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

### Authenticating your notebook environment
* If you are using **Colab** to run this notebook, uncomment the cell below and continue.
* If you are using **Vertex AI Workbench**, check out the setup instructions [here](https://github.com/GoogleCloudPlatform/generative-ai/tree/main/setup-env).

In [1]:
from google.colab import auth
auth.authenticate_user()

### Import libraries

**Colab only:** Uncomment the following cell to initialize the Vertex AI SDK. For Vertex AI Workbench, you don't need to run this.  

In [5]:
import vertexai

PROJECT_ID = "instantml-workshop-brlee"  # @param {type:"string"}
vertexai.init(project=PROJECT_ID, location="us-central1")

In [6]:
import pandas as pd
from vertexai.language_models import TextGenerationModel

### Import models

In [7]:
generation_model = TextGenerationModel.from_pretrained("text-bison@001")

## Question Answering

Question-answering capabilities require providing a prompt or a question that the model can use to generate a response. The prompt can be a few words or a few complete sentences, depending on the complexity of the question.

When creating a question-answering prompt, it is essential to be specific and provide as much context as possible. It helps the model understand the intent behind the question and generate a relevant response. For example, if you want to ask:

```
"What is the capital of France?",

then a good prompt could be:

"Please tell me the name of the city that serves as the capital of France."

```

In addition to being specific, the prompt should also be grammatically correct and free of spelling errors. It helps the model generate a response that is easy to understand and contains fewer errors or inaccuracies.

By providing specific, context-rich prompts, you can help the model understand the intent behind the question and generate accurate and relevant responses.


Below are some differences between the **open domain** and **closed domain** categories for question-answering prompts.

* **Open domain**: All questions whose answers are available online already. They can belong to any category, like history, geography, countries, politics, chemistry, etc. These include trivia or general knowledge questions, like:

```
Q: Who won the Olympic gold in swimming?
Q: Who is the President of [given country]?
Q: Who wrote [specific book]"?
```

Keep in mind the training cutoff of generative models, as questions involving information more recent than what the model was trained on might give incorrect or imaginative answers.


* **Closed domain**: If you have some internal knowledge base not available on the public internet, then those belong to the _closed domain_ category.
You can pass that "private" knowledge as context to the model. If prompted correctly, the model is more likely to answer from within the context provided and less likely to give answers beyond that from the open internet.

Consider the example of building a Q&A bot over your internal product documentation. In this case, you can pass the complete documentation to the model and prompt it only to answer based on that.

Typical prompt for **closed domain**:

```
Prompt: f""" Answer from the below context: \n\n
		   context: {your knowledge base} \n
		   question: {question specific to that knowledge base}  \n
		   answer: {to be predicted by model} \n
		"""
```

Below are some examples to understand these different types of prompts.

### Open Domain

#### Zero-shot prompting

In [21]:
prompt = """Q: Who was President of the United States in 1955? Which party did he belong to?\n
            A:
         """
print(
    generation_model.predict(
        prompt,
        max_output_tokens=256,
        temperature=0.1,
    ).text
)

Dwight D. Eisenhower, Republican


In [22]:
prompt = """Q: What is the tallest mountain in the world?\n
            A:
         """
print(
    generation_model.predict(
        prompt,
        max_output_tokens=20,
        temperature=0.1,
    ).text
)

Mount Everest


#### Few-shot prompting

Let's say you want to a get a short answer from the model (like only a specific name). To do so, you can leverage a few-shot prompt and provide examples to the model to illustrate the expected behavior.

In [20]:
prompt = """Q: Who is the current President of France?\n
            A: Emmanuel Macron \n\n

            Q: Who invented the telephone? \n
            A: Alexander Graham Bell \n\n

            Q: Who wrote the novel "1984"?
            A: George Orwell

            Q: Who discovered penicillin?
            A:
         """
print(
    generation_model.predict(
        prompt,
        max_output_tokens=20,
        temperature=0.1,
    ).text
)

Alexander Fleming


#### Zero-shot prompting vs Few-shot prompting

Zero-shot prompting can be useful for quickly generating text for new tasks, but the quality of the generated text may be lower than that of a few-shot prompt with well-chosen examples. Few-shot prompting is typically better suited for tasks that require a high degree of specificity or domain-specific knowledge, but requires some additional thought and potentially data to set up the prompt.

### Closed Domain

#### Adding internal knowledge as context in prompts

Imagine a scenario where you would like to build a question-answering bot that takes in internal documentation and lets users ask questions about it.

In the example below, the Google Cloud Storage and content policy documentation is added to the prompt, so that the PaLM API can use that to answer subsequent questions with the provided context.

In [8]:
context = """
Storage and content policy \n
How durable is my data in Cloud Storage? \n
Cloud Storage is designed for 99.999999999% (11 9's) annual durability, which is appropriate for even primary storage and
business-critical applications. This high durability level is achieved through erasure coding that stores data pieces redundantly
across multiple devices located in multiple availability zones.
Objects written to Cloud Storage must be redundantly stored in at least two different availability zones before the
write is acknowledged as successful. Checksums are stored and regularly revalidated to proactively verify that the data
integrity of all data at rest as well as to detect corruption of data in transit. If required, corrections are automatically
made using redundant data. Customers can optionally enable object versioning to add protection against accidental deletion.
"""

question = "How is high availability achieved?"

prompt = f"""Answer the question given in the contex below:
Context: {context}?\n
Question: {question} \n
Answer:
"""

print("[Prompt]")
print(prompt)

print("[Response]")
print(
    generation_model.predict(
        prompt,
    ).text
)

[Prompt]
Answer the question given in the contex below:
Context: 
Storage and content policy 

How durable is my data in Cloud Storage? 

Cloud Storage is designed for 99.999999999% (11 9's) annual durability, which is appropriate for even primary storage and
business-critical applications. This high durability level is achieved through erasure coding that stores data pieces redundantly
across multiple devices located in multiple availability zones.
Objects written to Cloud Storage must be redundantly stored in at least two different availability zones before the
write is acknowledged as successful. Checksums are stored and regularly revalidated to proactively verify that the data
integrity of all data at rest as well as to detect corruption of data in transit. If required, corrections are automatically
made using redundant data. Customers can optionally enable object versioning to add protection against accidental deletion.
?

Question: How is high availability achieved? 

Answer:

[R

#### Instruction-tuning outputs

Another way to help out language models is to provide additional instructions to frame the output in the prompt. To ensure the model doesn't respond to anything outside the context, the prompt can specify that the response should be "Information not available in provided context" if that's the case.

In [9]:
question = "What machined are required for hosting Vertex AI models?"
prompt = f"""Answer the question given the context below as {{Context:}}. \n
If the answer is not available in the {{Context:}} and you are not confident about the output,
please say "Information not available in provided context". \n\n
Context: {context}?\n
Question: {question} \n
Answer:
"""

print("[Prompt]")
print(prompt)

print("[Response]")
print(
    generation_model.predict(
        prompt,
        max_output_tokens=256,
        temperature=0.3,
    ).text
)

[Prompt]
Answer the question given the context below as {Context:}. 

If the answer is not available in the {Context:} and you are not confident about the output,
please say "Information not available in provided context". 


Context: 
Storage and content policy 

How durable is my data in Cloud Storage? 

Cloud Storage is designed for 99.999999999% (11 9's) annual durability, which is appropriate for even primary storage and
business-critical applications. This high durability level is achieved through erasure coding that stores data pieces redundantly
across multiple devices located in multiple availability zones.
Objects written to Cloud Storage must be redundantly stored in at least two different availability zones before the
write is acknowledged as successful. Checksums are stored and regularly revalidated to proactively verify that the data
integrity of all data at rest as well as to detect corruption of data in transit. If required, corrections are automatically
made using redu

#### Few-shot prompting

In [16]:
# Original 예제
prompt = """
Context:
The term "artificial intelligence" was first coined by John McCarthy in 1956. Since then, AI has developed into a vast
field with numerous applications, ranging from self-driving cars to virtual assistants like Siri and Alexa.

Question:
What is artificial intelligence?

Answer:
Artificial intelligence refers to the simulation of human intelligence in machines that are programmed to think and learn like humans.

---

Context:
The Wright brothers, Orville and Wilbur, were two American aviation pioneers who are credited with inventing and
building the world's first successful airplane and making the first controlled, powered and sustained heavier-than-air human flight,
 on December 17, 1903.

Question:
Who were the Wright brothers?

Answer:
The Wright brothers were American aviation pioneers who invented and built the world's first successful airplane
and made the first controlled, powered and sustained heavier-than-air human flight, on December 17, 1903.

---

Context:
The Mona Lisa is a 16th-century portrait painted by Leonardo da Vinci during the Italian Renaissance. It is one of
the most famous paintings in the world, known for the enigmatic smile of the woman depicted in the painting.

Question:
Who painted the Mona Lisa?

Answer:

"""
print(
    generation_model.predict(
        prompt,
    ).text
)

Leonardo da Vinci painted the Mona Lisa during the Italian Renaissance.


In [42]:
# 고객 예제 테스트 1. 카카오 예제 (주로 규정집 기반 답변) - 규정집의 각 카테고리 별 내용을 Context 에 넣기 (each shot 마다 규정집 항목으로 context 정의)
prompt = """
Context:
1. 채용 및 근로계약\n
1-1. 채용 \n
① 입사지원자는 이력서 1통, 자기소개서 1통을 제출해야 합니다.\n
② 회사는 입사지원자에게 신체적 조건(용모‧키‧체중 등), 출신지역‧혼인여부‧재산, 직계존비속 및 형제자매의 학력‧직업‧재산 등 직무수행에 필요하지 않은 사항은 채용심사 등 의 자료로 요구하지 않습니다. \n
1-2. 근로계약 \n
① 회사는 근로계약 체결시 다음 내용을 해당자에게 명확히 제시하고, 1~6번 내용이 포함된 근로계약서 1부를 근로계약을 체결한 사원에게 줍니다. *해당 사원의 동의 하에 이를 전자적 방법(이메일 등)으로 보낼 수 있습니다. \n
1. 임금의 구성항목, 계산방법, 지급방법 2. 소정근로시간, 휴게시간 3. 휴일 4. 연차유급휴가 5. 취업의 장소 및 종사해야 할 업무에 관한 사항 6. 근로계약기간(기간제사원에 한정) 7. 근로기준법 제93조제1호부터 제12호까지에 해당하는 내용 8. 근로기준법 제10장에 따른 기숙사에 관한 사항(기숙사가 있는 경우에 한정) \n
② 회사는 근로계약 체결 시 제1항의 일부 내용을 대신하기 위한 것임을 명확히 밝히며 해당 내용이 적시된 취업규칙을 제시·교부할 수 있습니다. \n
1-3. 수습기간 \n
① 신규채용자는 최초로 근무를 개시한 날부터 ㅇ개월간을 수습기간으로 합니다. \n
② 제1항의 수습기간은 근속년수에 포함하되, 수습시작일부터 3개월 이내의 기간은 평균임금산정기간에는 포함하지 않습니다. \n

Question:
근로계약

Answer:
① 회사는 근로계약 체결시 다음 내용을 해당자에게 명확히 제시하고, 1~6번 내용이 포함된 근로계약서 1부를 근로계약을 체결한 사원에게 줍니다. *해당 사원의 동의 하에 이를 전자적 방법(이메일 등)으로 보낼 수 있습니다. \n
1. 임금의 구성항목, 계산방법, 지급방법 2. 소정근로시간, 휴게시간 3. 휴일 4. 연차유급휴가 5. 취업의 장소 및 종사해야 할 업무에 관한 사항 6. 근로계약기간(기간제사원에 한정) 7. 근로기준법 제93조제1호부터 제12호까지에 해당하는 내용 8. 근로기준법 제10장에 따른 기숙사에 관한 사항(기숙사가 있는 경우에 한정) \n
② 회사는 근로계약 체결 시 제1항의 일부 내용을 대신하기 위한 것임을 명확히 밝히며 해당 내용이 적시된 취업규칙을 제시·교부할 수 있습니다. \n

Question:
채용 시 제출 서류

Answer:
입사지원자는 이력서 1통, 자기소개서 1통을 제출해야 합니다.\n

---

Context:
2. 복무 \n
2-1. 복무의무 \n
1. 사원은 맡은바 직무를 충실히 수행해야 합니다. 2. 사원은 직무상 알게 된 비밀을 엄수하고 회사기밀을 누설해서는 안됩니다. 다만, 공익신고자 보호법상의 ‘공익신고자’의 경우에는 적용되지 않습니다. 3. 사원은 회사의 규정을 준수하고 상사의 정당한 직무상 지시에 따라야 합니다. 4. 사원은 사원으로서 품위를 손상하거나 회사의 명예를 실추시키는 행위를 해서는 안됩니다. \n
2-2. 출근, 결근 \n
① 사원은 업무시간 시작 전까지 출근해 업무에 임할 준비를 해 정상적인 업무수행에 차질이 없도록 해야 합니다. \n
② 질병이나 그 밖의 부득이한 사유로 결근하고자 하는 경우에는 사전에 소속부서의 장의 승인을 받아야 합니다. 다만, 불가피한 사유로 사전에 승인을 받을 수 없는 경우 결근 당일에라도 그 사유를 명확히 해 사후 승인을 받아야 하며, 정당한 이유 없이 이러한 절차를 이행하지 않은 경우 무단결근 처리 됩니다. \n
2-3. 지각 ․ 조퇴 및 외출 \n
① 사원은 질병 그 밖의 부득이한 사유로 지각하게 되는 경우에는 사전에 부서의 장 또는 직근 상급자에게 알려야 하며, 부득이한 사정으로 사전에 알릴 수 없는 경우에는 사후에 라도 지체없이 이 사실을 알려야 합니다. \n
② 사원은 근로시간 중에는 사적인 용무를 이유로 근무 장소를 이탈할 수 없습니다. 다만, 질병이나 그 밖의 부득이한 사유가 있는 경우에는 소속부서의 장의 승인을 받아 조퇴 또는 외출할 수 있습니다. \n
③ 사원이 지각, 조퇴 또는 외출한 시간은 무급으로 처리함을 원칙으로 합니다. \n
2-4. 공민권행사 및 공의 직무 수행 \n
① 회사는 사원이 근무시간 중 선거권, 그 밖의 공민권을 행사하거나 공(公)의 직무를 수행하기 위하여 필요한 시간을 청구할 경우 이를 거부할 수 없으며, 그 시간은 유급으로 처리합니다. \n
② 회사는 제1항의 권리 행사나 공(公)의 직무를 수행하는데 지장이 없는 범위 내에서 사원이 청구한 시간을 변경할 수 있습니다. \n
2-5. 출장 \n
①회사는 업무수행을 위해 필요한 경우 사원에게 출장을 명할 수 있습니다. \n
② 회사는 행선지별 여비, 숙박비, 현지교통비 등 출장 비용을 실비 범위 내에서 지급합니다. \n

Question:
개인 사정으로 회사 결근

Answer:
① 사원은 업무시간 시작 전까지 출근해 업무에 임할 준비를 해 정상적인 업무수행에 차질이 없도록 해야 합니다. \n
② 질병이나 그 밖의 부득이한 사유로 결근하고자 하는 경우에는 사전에 소속부서의 장의 승인을 받아야 합니다. 다만, 불가피한 사유로 사전에 승인을 받을 수 없는 경우 결근 당일에라도 그 사유를 명확히 해 사후 승인을 받아야 하며, 정당한 이유 없이 이러한 절차를 이행하지 않은 경우 무 단결근 처리 됩니다. \n

Question:
몸이 아파서 회사 못 가면 무단 결근 처리 돼?

Answer:
질병이나 그 밖의 부득이한 사유로 결근하고자 하는 경우에는 사전에 소속부서의 장의 승인을 받아야 합니다. 다만, 불가피한 사유로 사전에 승인을 받을 수 없는 경우 결근 당일에라도 그 사유를 명확히 해 사후 승인을 받아야 하며, 정당한 이유 없이 이러한 절차를 이행하지 않은 경우 무 단결근 처리 됩니다. \n

---

Context:
3. 인사 \n
3-1. 인사위원회 인사위원회 메뉴를 선택하셨네요. 아래 항목 중 궁금하신 내용을 선택해 주세요. \n
3-1-1. 인사위원회의 구성 \n
① 인사위원회는 대표이사와 부서장 또는 그에 준하는 직급의 사원 중 대표이사가 임명하는 자로 총 5명 이내로 구성하되 근로자위원을 최소 1명 이상 포함되도록 합니다. \n
② 위원회의 위원장은 대표이사 또는 대표이사가 위임한 자로 합니다. \n
③ 위원회에는 인사(총무)담당자 1명을 간사로 둡니다. \n
3-1-2. 위원회의 기능 위원회는 사원의 표창·징계에 관한 사항, 그 밖에 사원의 인사에 관해 위원회의 의결이 필요한 사항에 대해 의결합니다. \n
3-1-3. 위원회의 소집 및 운영 \n
① 위원회는 의결사항이 있을 경우 위원장이 소집하며, 원칙적으로 회의 개최 7일 전 회의일시, 장소, 의제 등을 각 위원에게 통보해야 해요. \n
② 위원회는 재적위원 과반수의 출석과 출석위원 과반수의 찬성으로 의결해요. 다만, 징계에 관한 사항은 재적위원 3분의 2 이상의 찬성으로 의결해야 해요. \n
③ 위원장은 표결권을 가지며 찬성하는 수와 반대하는 수가 동일할 때에는 결정권을 가져요. \n
④ 위원회의 회의는 공개하지 않으며 회의내용과 관련된 사항은 누설해서는 안되요. 다만, 위원회의 의결로 공개할 수 있어요. \n
⑤ 위원회의 의결사항이 특정위원에 관한 사항을 의결할 때에는 그 위원은 그 건의 의결에 참여할 수 없어요. \n
3-2. 배치・전직 및 승진 \n
① 회사는 사원의 능력, 적성, 경력 등을 고려하여 부서의 배치, 전직, 승진 등 인사발령을 하며, 사원은 정당한 사유 없이 이를 거부할 수 없습니다. \n
② 회사는 제1항에 따른 인사발령을 할 때 합리적인 이유 없이 남녀를 차별하지 않습니다. \n
③ 인사발령의 기준 등 필요한 사항에 대하여는 별도의 규정으로 정합니다. \n
3-3. 휴직 및 복직\n
3-3-1. 휴직사유 및 기간\n
휴직하려는 사원은 휴직을 시작하려는 날의 30일 전까지 회사에 휴직원을 제출해야 합니다.\n
다음과 같은 사유로 휴직할 수 있으며, 3번 조항을 제외하고는 무급을 원칙으로 합니다.\n
3-3-2. 휴직명령\n
① 회사는 사원이 휴직원을 제출하면 이를 심사하여 휴직명령 여부를 결정하여 사원에게 서면으로 통보합니다.\n
② 육아휴직의 경우, 사원이 휴직을 시작하려는 날의 전날까지 계속 근로한 기간이 6개월 미만일 때는 휴직명령을 하지 않을 수 있습니다.\n
3-3-3. 준수사항\n
① 휴직자는 휴직기간 중 거주지의 변동 등의 사유가 있을 때에는 지체 없이 회사에 그 사실을 알려야 합니다.\n
② 회사는 사원이 육아휴직하는 경우 고용보험법령이 정하는 육아휴직급여를 받을 수 있도록 증빙서류를 제공하는 등 적극 협조합니다.\n
3-3-4. 복직\n
① 사원은 휴직기간 만료일 7일 전까지 복직원을 제출해야 해요. 다만, 휴직기간의 연장이 필요한 경우 휴직기간 만료일 30일 전까지 그 사유를 명시하여 승인을 신청해야 합니다.\n
② 회사는 신청일부터 ㅇ일 내에 휴직사유별 기간의 범위 내에서 휴직기간의 연장 승인 여부를 결정하여 서면으로 통보합니다.\n
③ 사원은 휴직기간 중 휴직사유가 소멸되었을 때에는 지체없이 복직원을 제출해야 합니다.\n
④ 회사는 휴직 중인 사원으로부터 복직원을 제출 받은 경우에는 최대한 빠른 시일 내에 휴직 전의 직무에 복직시키도록 노력하되, 부득이한 경우에는 그와 유사한 업무나 동등한 수준의 급여가 지급되는 직무로 복귀시키도록 노력합니다.\n
3-3-5. 근속기간의 계산 등\n
① 휴직기간은 근속기간에 넣되, 평균임금* 산정기준이 되는 기간에서는 제외합니다.\n
② 병역법에 따라 휴직한 기간은 근로자퇴직급여 보장법에 따른 퇴직금 산정을 위한 계속근로기간에서 제외합니다.\n
*“평균임금”이란 이를 산정하여야 할 사유가 발생한 날 이전 3개월 동안에 그 근로자에게 지급된 임금의 총액을 그 기간의 총일수로 나눈 금액을 말해요. 근로자가 취업한 후 3개월 미만인 경우도 이에 준합니다.(근로기준법)"\n

Question:
6개월 미만으로 일한 사원도 휴직할 수 있나요?

Answer:

"""
print(
    generation_model.predict(
        prompt,
    ).text
)
### 질문 예제
# 승진
# 인사위원회는 누가 임명해?
# 인사위원회
# 인사발령을 거부할 수 있어?
# 회사에서 육아휴직 거부할 수 있나요
# 6개월 미만으로 일한 사원도 휴직할 수 있나요

육아휴직의 경우, 사원이 휴직을 시작하려는 날의 전날까지 계속 근로한 기간이 6개월 미만일 때는 휴직명령을 하지 않을 수 있습니다.


In [35]:
question = "징계"
prompt = f"""Answer the question given the context below as {{Context:}}. \n
If the answer is not available in the {{Context:}} and you are not confident about the output,
please say "Information not available in provided context". \n\n
Context: {context}?\n
Question: {question} \n
Answer:
"""

print("[Prompt]")
print(prompt)

print("[Response]")
print(
    generation_model.predict(
        prompt,
        max_output_tokens=256,
        temperature=0.3,
    ).text
)

[Prompt]
Answer the question given the context below as {Context:}. 

If the answer is not available in the {Context:} and you are not confident about the output,
please say "Information not available in provided context". 


Context: 
출장과 관련하여, 회사는 업무수행을 위해 필요한 경우 사원에게 출장을 명할 수 있습니다. 또한 회사는 행선지별 여비, 숙박비, 현지교통비 등 출장 비용을 실비 범위 내에서 지급합니다.

비상시 임금 및 비용 지불과 관련하여, 사원이 다음 각 호의 사유로 청구하는 경우에는 지급기일 전이라도 이미 제공한 근로에 대한 임금을 지급합니다.

1. 사원 또는 그의 수입에 의하여 생활을 유지하는 자의 출산, 질병 또는 재해의 비용에 충당하는 경우

2. 사원 또는 그의 수입에 의하여 생활하는 자의 혼인 또는 사망 시 그 비용에 충당하는 경우

3. 사원이 부득이한 사정으로 1주일 이상 귀향하는 경우

?

Question: 징계 

Answer:

[Response]
Information not available in provided context


In [48]:
# 고객 예제 테스트 2. Turing 예제 (주로 규정집 기반 답변) - 규정집의 각 카테고리 별 내용을 Context 에 넣기 (each shot 마다 규정집 항목으로 context 정의)

prompt = """
Context:
주요 키워드 : 채용, 채용시 제출 서류 알려줘, 입사지원시 필요 서류, 채용심사 자료, 입사지원자가 제출해야 할 서류\n
해당하는 답변 내용 :\n
① 입사지원자는 이력서 1통, 자기소개서 1통을 제출해야 합니다.\n
② 회사는 입사지원자에게 신체적 조건(용모‧키‧체중 등), 출신지역‧혼인여부‧재산, 직계존비속 및 형제자매의 학력‧직업‧재산 등 직무수행에 필요하지 않은 사항은 채용심사 등의 자료로 요구하지 않습니다.\n

Question:
입사지원 서류

Answer:
입사지원자는 이력서 1통, 자기소개서 1통을 제출해야 합니다.\n

---
Context:
주요 키워드 : 근로계약, 근로계약 체결시 알려줘야 할 항목, 근로계약서 양식, 근로계약서 이메일로 보내줘도 돼?, 근로계약서 필수 포함항목\n
해당하는 답변 내용 :\n
① 회사는 근로계약 체결시 다음 내용을 해당자에게 명확히 제시하고, 1~6번 내용이 포함된 근로계약서 1부를 근로계약을 체결한 사원에게 줍니다. \n
*해당 사원의 동의 하에 이를 전자적 방법(이메일 등)으로 보낼 수 있습니다.\n
1. 임금의 구성항목, 계산방법, 지급방법\n
2. 소정근로시간, 휴게시간\n
3. 휴일\n
4. 연차유급휴가\n
5. 취업의 장소 및 종사해야 할 업무에 관한 사항\n
6. 근로계약기간(기간제사원에 한정)\n
7. 근로기준법 제93조제1호부터 제12호까지에 해당하는 내용\n
8. 근로기준법 제10장에 따른 기숙사에 관한 사항(기숙사가 있는 경우에 한정)\n
② 회사는 근로계약 체결 시 제1항의 일부 내용을 대신하기 위한 것임을 명확히 밝히며 해당 내용이 적시된 취업규칙을 제시·교부할 수 있습니다.\n

Question:
근로계약

Answer:
① 회사는 근로계약 체결시 다음 내용을 해당자에게 명확히 제시하고, 1~6번 내용이 포함된 근로계약서 1부를 근로계약을 체결한 사원에게 줍니다. \n
*해당 사원의 동의 하에 이를 전자적 방법(이메일 등)으로 보낼 수 있습니다.\n
1. 임금의 구성항목, 계산방법, 지급방법\n
2. 소정근로시간, 휴게시간\n
3. 휴일\n
4. 연차유급휴가\n
5. 취업의 장소 및 종사해야 할 업무에 관한 사항\n
6. 근로계약기간(기간제사원에 한정)\n
7. 근로기준법 제93조제1호부터 제12호까지에 해당하는 내용\n
8. 근로기준법 제10장에 따른 기숙사에 관한 사항(기숙사가 있는 경우에 한정)\n
② 회사는 근로계약 체결 시 제1항의 일부 내용을 대신하기 위한 것임을 명확히 밝히며 해당 내용이 적시된 취업규칙을 제시·교부할 수 있습니다.\n

---
Context:
주요 키워드 : 수습기간, 수습기간이 뭐야, 수습기간은 얼마나 돼, 수습기간도 평균임금산정기간에 포함 돼?, 수습기간도 근속년수에 포함 돼?, 수습기간 알려줘\n
해당하는 답변 내용 :\n
① 신규채용자는 최초로 근무를 개시한 날부터 ㅇ개월간을 수습기간으로 합니다. \n
② 제1항의 수습기간은 근속년수에 포함하되, 수습시작일부터 3개월 이내의 기간은 평균임금산정기간에는 포함하지 않습니다.\n

Question:
수습기간

Answer:
신규채용자는 최초로 근무를 개시한 날부터 ㅇ개월간을 수습기간으로 합니다. \n

---
Context:
주요 키워드 : 복무의무, 근무하면서 지켜야할 의무사항 있어, 근무하면서 꼭 지켜야 할 것, 근무시 태도, 회사에서 일할 때 태도, 공익신고자 보호, 업무상 기밀은 어떻게 해\n
해당하는 답변 내용 :\n
1. 사원은 맡은바 직무를 충실히 수행해야 합니다. \n
2. 사원은 직무상 알게 된 비밀을 엄수하고 회사기밀을 누설해서는 안됩니다. 다만, 공익신고자 보호법상의 ‘공익신고자’의 경우에는 적용되지 않습니다.\n
3. 사원은 회사의 규정을 준수하고 상사의 정당한 직무상 지시에 따라야 합니다.\n
4. 사원은 사원으로서 품위를 손상하거나 회사의 명예를 실추시키는 행위를 해서는 안됩니다. \n

Question:
근무하면서 지켜야 하는 사항

Answer:
1. 사원은 맡은바 직무를 충실히 수행해야 합니다. \n
2. 사원은 직무상 알게 된 비밀을 엄수하고 회사기밀을 누설해서는 안됩니다. 다만, 공익신고자 보호법상의 ‘공익신고자’의 경우에는 적용되지 않습니다.\n
3. 사원은 회사의 규정을 준수하고 상사의 정당한 직무상 지시에 따라야 합니다.\n
4. 사원은 사원으로서 품위를 손상하거나 회사의 명예를 실추시키는 행위를 해서는 안됩니다. \n

---
Context:
주요 키워드 : 출근, 결근, 미리 말 못하고 결근, 아파서 결근할 때, 무단결근 기준, 개인 사정으로 회사 결근, 몸이 아파서 회사 못 가면 무단 결근 처리 돼?\n
해당하는 답변 내용 :\n
① 사원은 업무시간 시작 전까지 출근해 업무에 임할 준비를 해 정상적인 업무수행에 차질이 없도록 해야 합니다.\n
② 질병이나 그 밖의 부득이한 사유로 결근하고자 하는 경우에는 사전에 소속부서의 장의 승인을 받아야 합니다. \n
다만, 불가피한 사유로 사전에 승인을 받을 수 없는 경우 결근 당일에라도 그 사유를 명확히 해 사후 승인을 받아야 하며, 정당한 이유 없이 이러한 절차를 이행하지 않은 경우 무단결근 처리 됩니다.\n

Question:
결근

Answer:
① 사원은 업무시간 시작 전까지 출근해 업무에 임할 준비를 해 정상적인 업무수행에 차질이 없도록 해야 합니다.\n
② 질병이나 그 밖의 부득이한 사유로 결근하고자 하는 경우에는 사전에 소속부서의 장의 승인을 받아야 합니다. \n
다만, 불가피한 사유로 사전에 승인을 받을 수 없는 경우 결근 당일에라도 그 사유를 명확히 해 사후 승인을 받아야 하며, 정당한 이유 없이 이러한 절차를 이행하지 않은 경우 무단결근 처리 됩니다.\n

---
Context:
주요 키워드 : 지각 ․ 조퇴 및 외출, 지각했을 때, 아파서 조퇴했는데 유급처리 되나요?, 지각, 조퇴, 외출한 시간은 임금에서 공제 되나요?, 외출한 시간은 무급이야?, 조퇴하면 무급으로 처리 되나요?\n
해당하는 답변 내용 :\n
① 사원은 질병 그 밖의 부득이한 사유로 지각하게 되는 경우에는 사전에 부서의 장 또는 직근 상급자에게 알려야 하며, 부득이한 사정으로 사전에 알릴 수 없는 경우에는 사후에라도 지체없이 이 사실을 알려야 합니다.\n
② 사원은 근로시간 중에는 사적인 용무를 이유로 근무 장소를 이탈할 수 없습니다. 다만, 질병이나 그 밖의 부득이한 사유가 있는 경우에는 소속부서의 장의 승인을 받아 조퇴 또는 외출할 수 있습니다.\n
③ 사원이 지각, 조퇴 또는 외출한 시간은 무급으로 처리함을 원칙으로 합니다. \n

Question:
조퇴 시 무급?

Answer:
사원이 지각, 조퇴 또는 외출한 시간은 무급으로 처리함을 원칙으로 합니다. \n

---
Context:
주요 키워드 : 공민권행사 및 공의 직무 수행, 선거일에 출근하면 중간에 선거하러 갈 수 있나요?, 선거 때문에 외출하면 유급 처리 되나요?, 예비군 때문에 결근하면 무급인가요?, 민방위 훈련시간은 유급이야?, 회사가 사원의 선거 시간을 지정할 수 있나요?        \n
해당하는 답변 내용 :\n
① 회사는 사원이 근무시간 중 선거권, 그 밖의 공민권을 행사하거나 공(公)의 직무를 수행하기 위하여 필요한 시간을 청구할 경우 이를 거부할 수 없으며, 그 시간은 유급으로 처리합니다.\n
② 회사는 제1항의 권리 행사나 공(公)의 직무를 수행하는데 지장이 없는 범위 내에서 사원이 청구한 시간을 변경할 수 있습니다.\n

Question:
예비군

Answer:
① 회사는 사원이 근무시간 중 선거권, 그 밖의 공민권을 행사하거나 공(公)의 직무를 수행하기 위하여 필요한 시간을 청구할 경우 이를 거부할 수 없으며, 그 시간은 유급으로 처리합니다.\n
② 회사는 제1항의 권리 행사나 공(公)의 직무를 수행하는데 지장이 없는 범위 내에서 사원이 청구한 시간을 변경할 수 있습니다.\n

---
Context:
주요 키워드 : 출장, 출장에 대해 알려줘, 출장가면 숙박비 지원 해줘?, 출장비 받을 수 있어?, 출장비 지급 규정, 출장 갈 때 돈\n
해당하는 답변 내용 :\n
① 회사는 업무수행을 위해 필요한 경우 사원에게 출장을 명할 수 있습니다.\n
② 회사는 행선지별 여비, 숙박비, 현지교통비 등 출장 비용을 실비 범위 내에서 지급합니다.\n

Question:
출장비

Answer:
회사는 행선지별 여비, 숙박비, 현지교통비 등 출장 비용을 실비 범위 내에서 지급합니다.\n

---
Context:
주요 키워드 : 인사위원회의 구성, 인사위원회는 어떻게 구성되나요?, 인사위원회 구성 시 꼭 알아야 할 사항, 인사위원회는 누가 임명해?, 인사위원회는 누구 누구로 이뤄져?, 인사위원회에 꼭 넣어야 하는 사람\n
해당하는 답변 내용 :\n
① 인사위원회는 대표이사와 부서장 또는 그에 준하는 직급의 사원 중 대표이사가 임명하는 자로 총 5명 이내로 구성하되 근로자위원을 최소 1명 이상 포함되도록 합니다.\n
② 위원회의 위원장은 대표이사 또는 대표이사가 위임한 자로 합니다. \n
③ 위원회에는 인사(총무)담당자 1명을 간사로 둡니다.\n

Question:
인사위원회 임명

Answer:
① 인사위원회는 대표이사와 부서장 또는 그에 준하는 직급의 사원 중 대표이사가 임명하는 자로 총 5명 이내로 구성하되 근로자위원을 최소 1명 이상 포함되도록 합니다.\n
② 위원회의 위원장은 대표이사 또는 대표이사가 위임한 자로 합니다. \n
③ 위원회에는 인사(총무)담당자 1명을 간사로 둡니다.\n
---
주요 키워드 : 배치・전직 및 승진, 인사발령, 인사발령 기준, 인사발령 관련 규정 알려줘, 승진 기준 알려줘, 인사발령 거부할 수 있나요\n
해당하는 답변 내용 :\n
① 회사는 사원의 능력, 적성, 경력 등을 고려하여 부서의 배치, 전직, 승진 등 인사발령을 하며, 사원은 정당한 사유 없이 이를 거부할 수 없습니다.\n
② 회사는 제1항에 따른 인사발령을 할 때 합리적인 이유 없이 남녀를 차별하지 않습니다.\n
③ 인사발령의 기준 등 필요한 사항에 대하여는 별도의 규정으로 정합니다.\n


Question:
인사발령에 필요한 사항이 뭐야?

Answer:

"""
print(
    generation_model.predict(
        prompt,
    ).text
)

## 질문 예제
# 인사 발령 거부 가능할까?
# 남녀 차별
# 배치
# 인사발령에 필요한 사항이 뭐야?

① 회사는 사원의 능력, 적성, 경력 등을 고려하여 부서의 배치, 전직, 승진 등 인사발령을 하며, 사원은 정당한 사유 없이 이를 거부할 수 없습니다.

② 회사는 제1항에 따른 인사발령을 할 때 합리적인 이유 없이 남녀를 차별하지 않습니다.

③ 인사발령의 기준 등 필요한 사항에 대하여는 별도의 규정으로 정합니다.




### Extractive Question-Answering

In the next example, the generative model is guided to understand the meaning of the question and the passage, and to identify the relevant information in the passage that answers the question. The model is given a question and a passage of text, and is asked to find the answer to the question within the passage. The answer is typically a phrase or sentence.

In [19]:
prompt = """
Background: There is evidence that there have been significant changes in Amazon rainforest vegetation over the last 21,000 years through the Last Glacial Maximum (LGM) and subsequent deglaciation.
Analyses of sediment deposits from Amazon basin paleo lakes and from the Amazon Fan indicate that rainfall in the basin during the LGM was lower than for the present, and this was almost certainly
associated with reduced moist tropical vegetation cover in the basin. There is debate, however, over how extensive this reduction was. Some scientists argue that the rainforest was reduced to small,
isolated refugia separated by open forest and grassland; other scientists argue that the rainforest remained largely intact but extended less far to the north, south, and east than is seen today.
This debate has proved difficult to resolve because the practical limitations of working in the rainforest mean that data sampling is biased away from the center of the Amazon basin, and both
explanations are reasonably well supported by the available data.

Q: What does LGM stands for?
A: Last Glacial Maximum.

Q: What did the analysis from the sediment deposits indicate?
A: Rainfall in the basin during the LGM was lower than for the present.

Q: What are some of scientists arguments?
A: The rainforest was reduced to small, isolated refugia separated by open forest and grassland.

Q: There have been major changes in Amazon rainforest vegetation over the last how many years?
A: 21,000.

Q: What caused changes in the Amazon rainforest vegetation?
A: The Last Glacial Maximum (LGM) and subsequent deglaciation

Q: What has been analyzed to compare Amazon rainfall in the past and present?
A: Sediment deposits.

Q: What has the lower rainfall in the Amazon during the LGM been attributed to?
A:
"""

print(
    generation_model.predict(
        prompt,
    ).text
)

Reduced moist tropical vegetation cover in the basin.


### Evaluation

You can evaluate the outputs of the question and answering task if the ground truth answers of each question are available. In zero-shot prompting, you can only use `open domain` questions. However, with `closed domain` questions, you can add context and evaluate similarly.  To showcase how that will work, start by creating a simple dataframe with questions and ground truth answers.

In [11]:
qa_data = {
    "question": [
        "In a website browser address bar, what does “www” stand for?",
        "Who was the first woman to win a Nobel Prize",
        "What is the name of the Earth’s largest ocean?",
    ],
    "answer_groundtruth": ["World Wide Web", "Marie Curie", "The Pacific Ocean"],
}
qa_data_df = pd.DataFrame(qa_data)
qa_data_df

,question,answer_groundtruth
0,"In a website browser address bar, what does “w...",World Wide Web
1,Who was the first woman to win a Nobel Prize,Marie Curie
2,What is the name of the Earth’s largest ocean?,The Pacific Ocean


In [29]:
qa_data = {
    "question": [
        "산업재해",
        "출장",
        "교통",
    ]
}
qa_data_df = pd.DataFrame(qa_data)
qa_data_df

,question
0,산업재해
1,출장
2,교통


Now that you have the data with questions and ground truth answers, you can call the PaLM 2 generation model to each review row using the `apply` function. Each row will use the dynamic prompt to predict the answer using the PaLM API. We will save the results in `answer_prediction` column.  


In [30]:
def get_answer(row):
    prompt = f"""Answer the following question as precise as possible.\n\n
            question: {row}
            answer:
              """
    return generation_model.predict(
        prompt=prompt,
    ).text


qa_data_df["answer_prediction"] = qa_data_df["question"].apply(get_answer)
qa_data_df

,question,answer_prediction
0,산업재해,산업재해(industrial accident)는 산업 활동 중 발생하는 사고를 말한다.
1,출장,출장 (Korean)
2,교통,교통은 움직이는 물체의 흐름을 말한다.


You may want to evaluate the answers predicted by the PaLM API. However, it will be more complex than the text classification since the answers may differ from ground truth and may be presented in slightly more/fewer words.

For example, you can observe the question "What is the name of the Earth's largest ocean?" and see that model predicted  "Pacific Ocean" when a ground truth label is "The Pacific Ocean" with the extra "The." Now, if you use the simple classification metrics, then you will consider this as a wrong prediction since original and predicted strings have a difference. However, you can see that the answer is correct since an extra "The" is causing the issue. It's a simple string comparison problem.

The solution to string comparison where both `ground_thruth` and `predicted` may have some extra or fewer letters, one approach is to use a fuzzy matching algorithm.
Fuzzy string matching uses [Levenshtein Distance](https://en.wikipedia.org/wiki/Levenshtein_distance) to calculate the differences between two strings.

For example, the Levenshtein distance between "kitten" and "sitting" is 3, since the following 3 edits change one into the other, and there is no way to do it with fewer than 3 edits:

* kitten → sitten (substitution of "s" for "k"),
* sitten → sittin (substitution of "i" for "e"),
* sittin → sitting (insertion of "g" at the end).


Here's another example, but this time using `fuzzywuzzy`  library, which gives us the same `Levenshtein distance` between two strings but in ratio. The ratio raw score measures the string's similarity as an int in the range [0, 100]. For two strings X and Y, the score is defined by int(round((2.0 * M / T) * 100)) where T is the total number of characters in both strings, and M is the number of matches in the two strings.

Read more here about the [ratio formula](https://anhaidgroup.github.io/py_stringmatching/v0.3.x/Ratio.html) :

You can see one example to understand this furhter.
```
String1: "this is a test"
String2: "this is a test!"

Fuzz Ratio => 97  #

Fuzz Partial Ratio => 100  #Since most characters are the same and in a similar sequence, the algorithm calculates the partial ratio as 100 and ignores simple additions (new characters).
```


First, install the package `fuzzywuzzy` and `python-Levenshtein`:

In [10]:
!pip install -q python-Levenshtein --upgrade --user
!pip install -q fuzzywuzzy --upgrade --user

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 53.6 MB/s eta 0:00:00


Then compute a score to perform fuzzy matching:

In [13]:
from fuzzywuzzy import fuzz


def get_fuzzy_match(df):
    return fuzz.partial_ratio(df["answer_groundtruth"], df["answer_prediction"])


qa_data_df["match_score"] = qa_data_df.apply(get_fuzzy_match, axis=1)
qa_data_df

,question,answer_groundtruth,answer_prediction,match_score
0,"In a website browser address bar, what does “w...",World Wide Web,World Wide Web,100
1,Who was the first woman to win a Nobel Prize,Marie Curie,Marie Curie,100
2,What is the name of the Earth’s largest ocean?,The Pacific Ocean,Pacific Ocean,100


Now that you have the individual match score (partial), you can take the mean or average of the whole column to get a sense of overall data.
Scores closer to 100 mean PaLM 2 can predict closer to ground truth; if the score is towards 50 or 0, it did not perform well.

In [14]:
print(
    "the average match score of all predicted answer from PaLM 2 is : ",
    qa_data_df["match_score"].mean(),
    " %",
)

the average match score of all predicted answer from PaLM 2 is :  100.0  %


In this case, you get 100% as the mean score, even though some predictions were missing some words. That means you are very close to the ground truth, and some answers are just missing the exact verboseness of the ground truth.